# JuMP: Diagnosing infeasible models

Notebooks 1 and 2 define basic objects in JuMP such as variables, constraints, and expressions. This tutorial builds further to discuss how to diagnose infeasible models.

## Install Gurobi

Diagnosing infeasibilities is much easier with an advanced solver such as Gurobi, which has the ability to find infeasible constraints for you. A solver such as Gurobi will also be helpful for solving larger models later in the class, and for your final project.

Let's start by installing a Gurobi academic license. To start, first go to https://www.gurobi.com and click "Register" at the top right to create an account. Follow through the steps to confirm your account. Then, once you are logged in, click "Licenses", click "Request", and generate a "Named-User Academic" license. Follow the steps to install the license. Once a license is installed, you can run "gurobi_cl" in the command line to check that the license was installed correctly.

Next, open Julia and install the Gurobi package. Once it is installed, you can continue with the rest of this notebook.

## Diagnosing an infeasible model

Let's start with the same factory optimization example from notebook 2. We are going to purposefully add an infeasible constraint, and then figure out how to diagnose it.

In [46]:
using JuMP
using Gurobi

In [47]:
factory_model = Model(Gurobi.Optimizer);
@variables(factory_model, begin
    8 >= x >= 0
    y >= 0
    end)
@constraints(factory_model, begin
    time_constraint, 10x + 8y <= 80
    materials_constraint, 7x + 11y <= 77    
    end)
@expression(factory_model, objective, 150x + 175y)
@objective(factory_model, Max, objective)
print(factory_model)


Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-14


In [42]:
optimize!(factory_model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x3b4095a3
Coefficient statistics:
  Matrix range     [7e+00, 1e+01]
  Objective range  [2e+02, 2e+02]
  Bounds range     [8e+00, 8e+00]
  RHS range        [8e+01, 8e+01]
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7500000e+32   1.187500e+30   1.750000e+02      0s
       2    1.4138889e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.413888889e+03

User-callback calls 47, time in user-callback 0.00 sec


In [43]:
value.(x), value.(y)

(4.888888888888888, 3.888888888888889)

Now, let's try adding a constraint that is clearly in conflict with the constraint x<=8. What happens when we solve the model?

In [44]:
@constraint(factory_model,test_constraint,x>=10) ;

print(factory_model)

In [45]:
optimize!(factory_model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+02, 2e+02]
  Bounds range     [8e+00, 8e+00]
  RHS range        [1e+01, 8e+01]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4138889e+03   5.111111e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Infeasible model

User-callback calls 20, time in user-callback 0.00 sec


### Diagnosing the problem

Gurobi is reporting that we have an "Infeasible model." This means that there is nowhere in the space of decision variables where all the constraints are obeyed. We can figure out which constraints in particular are causing the infeasibility by running "compute_conflict!" on the model, and printing out the results:

In [39]:
compute_conflict!(factory_model) ;

if get_attribute(factory_model, MOI.ConflictStatus()) == MOI.CONFLICT_FOUND
    iis_model, _ = copy_conflict(factory_model)
    print(iis_model)
end


IIS computed: 1 constraints and 1 bounds
IIS runtime: 0.00 seconds (0.00 work units)


Voila! Gurobi has identified the set of constraints that are leading to the infeasibility. In this context it was obvious where the infeasibility came from, but in larger models it can be quite difficult to identify what you did wrong, which is why this method is helpful.